Run several times a week. Imports a table from the database containing USPS zone information, then combining that information with catalog orders in a spreadsheet.

In [1]:
#second iteration after migration; script combining information with a zones table uploaded to the reports database
#saved at least 50% in catalog shipment costs, run several times a week

import warnings

warnings.filterwarnings('ignore')

import pyodbc 

cnxn = pyodbc.connect(driver = '{ODBC Driver 17 for SQL Server}', server = 'tcp:10.3.6.50', database = 'reports', trusted_connection = 'yes')

cursor = cnxn.cursor()

import pandas as pd

query = '''

SELECT *

FROM USPS_Zones

'''

dfUSPSZones = pd.read_sql(query, cnxn)

dfCustomerAddresses = pd.read_excel(r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Catalogs\Files\Catalogs.xlsx')

dfCustomerAddresses = dfCustomerAddresses.fillna('')

dfCustomerAddresses = dfCustomerAddresses[((dfCustomerAddresses.Shippped == '') & (dfCustomerAddresses['Non-US'] == '') & (dfCustomerAddresses['postal_code'].astype(str).str.isnumeric()))]

dfJoined = dfCustomerAddresses.merge(dfUSPSZones, how = 'cross')

dfJoined['ZIPCODE_NUMERIC'] = dfJoined['postal_code'].astype(str).str.slice(0,5)

dfJoined[['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC', 'Zone']] = dfJoined[['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC', 'Zone']].apply(pd.to_numeric)

dfJoined = dfJoined.query('ZipCode_FROM <= ZIPCODE_NUMERIC <= ZipCode_TO')

dfJoined['Zone'] = dfJoined['Zone'].fillna(0)

dfJoined['Zone'] = dfJoined['Zone'].astype(int)

dfJoined['quantity_ordered'] = dfJoined['quantity_ordered'].astype(int)

dfJoined.drop(['ZipCode_FROM', 'ZipCode_TO', 'ZIPCODE_NUMERIC', 'sku', 'product_name', 'Shippped', 'Non-US'], inplace=True, axis=1)

dfJoined = dfJoined.rename({'Zone': 'ZONE'}, axis = 1)

dfJoined.reset_index(drop=True, inplace=True)

dfJoined.to_excel('WeeklyCatalogShipments.xlsx', index=False, header=True)

import shutil

original = r'C:\Users\USER\OneDrive - COMPANY\Code\Python\Jupyter\DataProjects\CatalogShipments\WeeklyCatalogShipments.xlsx'
target = r'C:\Users\USER\OneDrive - COMPANY\Data Projects\Catalogs\WeeklyCatalogShipments.xlsx'

shutil.copyfile(original, target)

print('Done.')

Done.
